In [1]:
import ee
import geemap
from pathlib import Path

In [2]:
ee.Authenticate()

Enter verification code:  4/1ASVgi3JZTyR0W4SJcrM-nXYzt2-dv3LTNhuxReCBrz--uXXv1x486eRI2FI



Successfully saved authorization token.


In [3]:
ee.Initialize()

Création d'une ee.Geometry() contenant la région Aquitaine (1 seul polygone)

In [4]:
ROI_FILE = Path("aquitaine.geojson")

roi = geemap.geojson_to_ee(str(ROI_FILE))

In [5]:
map = geemap.Map()
# map.add_basemap("Esri.WorldImagery")
map.add_basemap("Stadia.AlidadeSmoothDark")
map.centerObject(roi)

In [6]:
DIST_ALERT_RESOURCE = "projects/glad/HLSDIST/current"
veg_dist_status = ee.ImageCollection(f"{DIST_ALERT_RESOURCE}/VEG-DIST-STATUS").mosaic()

In [7]:
veg_dist_confirmed_alerts_mask = veg_dist_status.where(veg_dist_status.eq(6).Or(veg_dist_status.eq(8)), 1)
veg_dist_confirmed_alerts = veg_dist_confirmed_alerts_mask.selfMask()

In [8]:
roi_fc = ee.FeatureCollection(roi)
empty_img = ee.Image().byte()
roi_contour = empty_img.paint(**{"featureCollection": roi_fc, "color": 1, "width": 3})
map.addLayer(roi_contour, {"palette": "blue"}, "Aquitaine")

In [9]:
map.addLayer(veg_dist_confirmed_alerts, {"min": 6, "max": 8, "palette": ["orange", "red"]}, "VEG-DIST-CONFIRMED-ALERTS")

In [10]:
map

Map(center=[44.97663510825801, 0.40994910693179953], controls=(WidgetControl(options=['position', 'transparent…